In [38]:
-- tables in this DB are category; date; event; listing; sales; users; venue

-- Preview the tables in the data sets: 

SELECT * 
FROM sales -- I changed this to each table name to look at the tables
LIMIT (10);

-- eventid is a field in event & listing
-- venueid is a field in venue, event
-- catid is a field in category, event, 


**Some notes about this database that might be helpful for my reference:**

* eventid = unique identifier for each event; found in the tables **event** and **listing**. 
* venueid = unique identifier for each venue; found in the tables **venue** and **event**
* dateid = unique identified for each date where a sale was made; found in the tables **sales** and **event** and **date**

**The questions I want to answer using this database include:**
1. On which non-holiday day of the year was the number of sales made the highest? 
2. Which category name (e.g. NHL, pop performances etc) were most popular? 
3. Which month were the most NFL tickets sold in? 
4. Which events were the top 10 for ticket revenue in the first quarter of 2008? 
5. Of the pop artists that performed in October, which one sold the largest number of tickets in total? 

The next block will query to find the day with the largest number of sales. 

In [27]:
SELECT 
	SUM(s.qtysold) AS num_tickets_sold, -- total number of tickets sold regardless of how many sales they were split into 
	COUNT(DISTINCT s.salesid) AS num_individual_sales, -- an individual sale may have bought multiple tickets, or just one
	d.caldate -- to identify the actual dates
FROM sales AS s
INNER JOIN date AS d -- aliased both tables for ease of writing out
ON s.dateid = d.dateid
WHERE d.holiday = 'False'
GROUP BY s.dateid, d.caldate
ORDER BY num_individual_sales DESC
LIMIT (10);

,num_tickets_sold,num_individual_sales,caldate
0,1158,597,2008-10-17 00:00:00+00:00
1,1170,586,2008-02-20 00:00:00+00:00
2,1187,585,2008-10-23 00:00:00+00:00
3,1133,583,2008-04-14 00:00:00+00:00
4,1216,581,2008-07-20 00:00:00+00:00
5,1129,580,2008-03-08 00:00:00+00:00
6,1173,578,2008-09-28 00:00:00+00:00
7,1198,574,2008-09-10 00:00:00+00:00
8,1152,574,2008-10-16 00:00:00+00:00
9,1147,574,2008-08-21 00:00:00+00:00


From the code above, it reveals that of the top 10 dates for individual sales: 

* 3/10 days were in October 
* 2/10 days were in September 
* The other five days were each in different months
* None of those dates were public holidays 

In the case of the numbers of tickets sold, for the top selling date (first record in results table: October 17th) there were fewer tickets sold than on the second-most popular selling date. This does _not_ pose a problem since the larger number of sales must include a lower mean number of tickets sold on each of those days. 

The next query will identify data to answer Q2: Which category name (e.g. NHL, pop performances etc) were most popular? 

_To answer this question, I will join the sales, event and category tables. This means I can find the number of sales for each event (from sales), identify the category ID ('catid') from the event table, then identify which category that falls under using the category table._

In [9]:
SELECT COUNT(DISTINCT s.salesid) AS num_sales, c.catname AS category_name
FROM event AS e
FULL JOIN sales as s
USING (eventid)
FULL JOIN category AS c
USING (catid)
GROUP BY catname


,num_sales,category_name
0,25737,Musicals
1,39223,Plays
2,97582,Pop
3,9914,Opera
4,0,Jazz
5,0,NBA
6,0,MLS
7,0,MLB
8,0,NFL
9,0,NHL


As the results of the above query shows, it looks like the most popular category was Pop (music), followed by Plays, Musicals and then Opera. 

The table also shows that there were 0 sales for classical or jazz music, and no sales for any sports at all. I want to see if I can find any sales from events that fall into any of the '0' count categories by adapting the above query. _Whilst I showed previews of all tables before starting analysis / deciding on questions using the first query and changing it, I did not notice catid values between 6 and 9 only._

To achieve this, I will copy in the above query and then add a condition to ensure that the catid is between 1 and 5 or 10 or 11, so that I'm only looking at the remaining categories. 

In [28]:
SELECT 
	COUNT(DISTINCT s.salesid) AS num_sales, 
	c.catid AS category_id
FROM event AS e
FULL JOIN sales as s
USING (eventid)
FULL JOIN category AS c
USING (catid)
GROUP BY e.catid, c.catid
HAVING e.catid IN (1, 2, 3, 4, 5, 10, 11) 


,num_sales,category_id


The above code returned "Your query ran successfully but returned no results". Since this is a publicly available dataset that once could assume had been artificially generated, this is a reasonable outcome. However, this means that Q3 was now unanswerable since there were no existing records associated with any sports. 

As such, the following question will be asked instead: 

**Q3 v2: Which month were the most Musical tickets sold in?**

The next block of code will query the results relating to tickets to musicals. 

This requires me to consider: 
* The _sales_ table includes an eventid but nothing else relating to category, and includes a dateid but no identification of the month
* The _event_ table includes the eventid and category id ('catid')

Process I'll follow: 
1. Join sales and event on eventid, and join sales with date on dateid
2. Filter for catid=6
3. Count the quantity sold ('qtysold') and return the month with it 
4. Group the results by month from the 'date' table, and order the results by number of tickets sold in descending order

In [34]:
SELECT 
	COUNT(qtysold) AS num_tickets_sold, 
	d.month
FROM sales AS s
LEFT JOIN event AS e
ON s.eventid = e.eventid
LEFT JOIN date AS d
ON s.dateid = d.dateid
WHERE catid = 6
GROUP BY d.month
ORDER BY num_tickets_sold DESC;


,num_tickets_sold,month
0,2670,OCT
1,2538,MAR
2,2519,AUG
3,2495,SEP
4,2433,MAY
5,2300,FEB
6,2262,JUN
7,2217,APR
8,2217,JUL
9,2090,NOV


As this result shows, the most tickets were sold in October, but this was very closely followed by March. By contrast, relatively very few tickets were sold in January and December each. It is unclear from the data available whether this is because people have been spending more money on usual due to the holidays (and therefore have less money to spend), or whether there are fewer musicals playing during this period. 

Finally, I will address **Q4: Which events were the top 10 for ticket revenue in 2008?**

Process to follow: 
1. Count the sum of total revenue (quantity sold * price paid in each record) after grouping by eventid. This involves joining table 'sales' and table 'event'
2. Find the 'eventname' from 'events' to make it obvious what the actual event is 
3. Sort the results by their total ticket revenue in descending order. 

In [3]:
SELECT 
	DISTINCT e.eventid, -- the results were the same without the DISTINCT keyword but still worth specifying
	e.eventname, -- not DISTINCT because multiple events have the same name
	SUM(s.qtysold * s.pricepaid) AS total_ticket_rev, -- included only one alias since the others are intuitively named
		-- needs to be SUM so that I can see the total ticket revenue, not just the ticket revenue for any individual record (which returns 			an error since there are multiple sales for each individual event)
	SUM(s.qtysold) AS num_tickets_sold -- this shows how many tickets were actually sold for any given event
FROM sales AS s
LEFT JOIN event AS e
ON s.eventid = e.eventid
LEFT JOIN date AS d
ON s.dateid = d.dateid
WHERE d.qtr = 1
GROUP BY e.eventid, e.eventname 
ORDER BY total_ticket_rev DESC -- want to see value of total ticket revenue, regardless of how many tickets were sold. 
LIMIT (10); 

,eventid,eventname,total_ticket_rev,num_tickets_sold
0,7177,At The Gates,157978.0,88
1,851,The Little Mermaid,138204.0,105
2,6044,Huey Lewis and the News,137248.0,69
3,7851,Live,116966.0,49
4,6845,Gordon Lightfoot,106916.0,98
5,2801,A Doll's House,106378.0,37
6,1771,Legally Blonde,106321.0,64
7,1920,The Tempest,99328.0,85
8,7513,Steve Miller Band,98627.0,64
9,5022,Counting Crows and Maroon 5,98307.0,64


As this result demonstrates, the event with the top revenue was At The Gates, producing $157,978 from 88 tickets. It felt odd that this was the revenue given that 88 is not a factor of 157,978 but running a new exploratory query (seen below) identifying the price per ticket for a given eventid, tickets sold for varying prices depending on where they were listed and who sold them. (For example, see records with indices 5-6 or 8-9, where the records have the same eventid and same dateid but different ticket prices).

In [66]:
SELECT *
FROM listing
ORDER BY eventid, dateid
LIMIT (10);

,listid,sellerid,eventid,dateid,numtickets,priceperticket,totalprice,listtime
0,58222,47089,1,1829,18,259.0,4662.0,2008-01-03 11:23:01+00:00
1,38357,3273,1,1833,4,115.0,460.0,2008-01-07 03:55:34+00:00
2,17540,48643,1,1835,10,149.0,1490.0,2008-01-09 01:14:15+00:00
3,889,1830,1,1837,7,50.0,350.0,2008-01-11 10:57:48+00:00
4,177000,43508,1,1844,12,40.0,480.0,2008-01-18 04:31:46+00:00
5,87056,21356,1,1849,14,140.0,1960.0,2008-01-23 10:34:17+00:00
6,3721,38855,1,1849,8,78.0,624.0,2008-01-23 12:58:03+00:00
7,120503,1064,2,2063,16,197.0,3152.0,2008-08-25 12:06:02+00:00
8,103622,19217,2,2068,8,496.0,3968.0,2008-08-30 09:52:23+00:00
9,136146,36118,2,2068,16,389.0,6224.0,2008-08-30 02:08:46+00:00


**Finally**, this last query will answer the question: of the pop artists that performed, which one sold the largest number of tickets in total in October? 

**Process**:
1. Filter the 'sales' table using sub-queries to only include pop events (catid = 9) and event month in October
2. Group the event by eventname (since this dictates the artist name) and count the number of tickets sold (qtysold)


In [26]:
SELECT e.eventname, SUM(s.qtysold) AS total_tix_sold
FROM event AS e
LEFT JOIN sales AS s
ON e.eventid = s.eventid
LEFT JOIN date AS d
ON e.dateid = d.dateid
WHERE 
	e.catid = 9 AND
	d.month = 'OCT'
GROUP BY eventname
ORDER BY total_tix_sold DESC
LIMIT (10);


,eventname,total_tix_sold
0,Built To Spill,223
1,Suzanne Vega,193
2,Scorpions,191
3,Chaka Khan,183
4,Jason Aldean,168
5,Cafe Tacuba,164
6,Nick Cave,160
7,Celtic Thunder,153
8,Bruce Hornsby,153
9,Duran Duran,147


As this final result shows, the pop band **Built To Spill** sold the most tickets for an October event at 223 tickets. 

Changing the month filter to different months showed similar ticket sales rates but for different bands, so given the data in the database, the results from this query are believable. 